<html>
    <body>
            <p> We will first start by importing the libraries required and import our dataset. The data may have some empty fields and those will need to be removed to get a better model. The dataset used is the following : <a href="https://www.kaggle.com/datasets/jruvika/fake-news-detection"> fake_news.csv </a></p>
    </body>
</html>

In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split

In [2]:
#save data in dataframe
df = pd.read_csv('data.csv')

In [3]:
#keep just label, headline and body
df = df[['Headline' , 'Body' , 'Label']].copy()
df.head()

,Headline,Body,Label
0,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [4]:
#remove empty rows
df = df.dropna()

In [8]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=7)

In [9]:
#tokenizes each word
tokenized_text = [text.split() for text in df['Body']]
#creates a new word2vec model with size of 100
word2vec_model = gensim.models.Word2Vec(tokenized_text, vector_size=200, window=10, min_count=20)

In [10]:
#function to get average word vector representation
#takes as input the list, pretrained model , unique words in models vocab and dimensionality of vectors
def average_word_vectors(words, model, vocabulary, num_features):
    #creates a vector with 0s of the lenght of numfeatures
    feature_vector = np.zeros((num_features,), dtype="float64")
    #nwords keep track of words
    nwords = 0.
    #iterate over list
    for word in words:
        #if word is in model, increase nword
        if word in vocabulary:
            nwords += 1
            #add the word to the vector with 0s
            feature_vector = np.add(feature_vector, model.wv[word])
    #check if nwords is now greater than 0 and if so calculate average
    if nwords > 0:
        feature_vector = np.divide(feature_vector, nwords)
    #return the vector
    return feature_vector

In [11]:
# Extract features for training data
#empty array to store
train_data = []
#iterates over train df
for _, row in train_df.iterrows():
    #apply the function of text column and adds it to the array
    features= average_word_vectors(row['Body'].split(), word2vec_model, word2vec_model.wv.key_to_index, 200)
    train_data.append(features)
#make a new array with labels of train_df
train_labels = train_df['Label']

In [12]:
# Repeat process for test data
#empty array to store
test_data = []
#iterates over train df
for _, row in test_df.iterrows():
    #apply the function of text column and adds it to the array
    features= average_word_vectors(row['Body'].split(), word2vec_model, word2vec_model.wv.key_to_index, 200)
    test_data.append(features)
#make a new array with labels of test_df
test_labels = test_df['Label']